In [4]:
%matplotlib notebook
from utils.dataloader import PascalVOCDataset
from utils.dataloader import myimshow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision as tv
import numpy as np
from model.rfcn_resnet101 import RFCNResnet101
from trainer import FasterRCNNTrainer
import matplotlib.pyplot as plt
from utils.vis_tool import vis_bbox
from utils import array_tool as at


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Download trained model from drive: https://drive.google.com/open?id=1AypEpNjTRVekSp4vQ-mwI07hSUQsH-HJ

In [3]:
''' After model is downloaded into local checkpoints folder, run to load model into trainer'''
rfcn = RFCNResnet101()
trainer = FasterRCNNTrainer(rfcn).to(device)
trainer.load('checkpoints/rfcn_06091445')

/datasets/home/home-02/50/150/prvo/Multi-Object-Detection/model/rfcn_resnet101.py:83: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.cls_layer.weight.data, 0.0, 0.01)
/datasets/home/home-02/50/150/prvo/Multi-Object-Detection/model/rfcn_resnet101.py:87: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.reg_layer.weight.data, 0.0, 0.01)


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/rfcn_06091445'

In [13]:
'''Get image from validation loader to get. Although attribute is faster_rcnn, it is the rfcn model passed from previous cells'''
img, bboxes, lbls, scale = next(iter(val_loader))
img = img.to(device)
trainer.faster_rcnn.predict(img)

([], [], [])

Our network failed to learn to represent objects or there are other issues that could not be solved in time